In [ ]:
#MODELO DE REGRESSÃO GERAL - MODELO DE AFINAÇÃO DOS PARÂMETROS DAS META-HEURÍSTICAS ------------------------------------------------------------------

import pandas as pd
import os
import random
import math
import numpy as np 
from matplotlib import pyplot as plt
import xlwings as xw
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor

#EXCEL - CAMINHO DO FICHEIRO
file_path = r"D:\Documentos\...\DataSet_Regressão_ModeloGeral.xlsx"
dados_problema= pd.read_excel(file_path)
print (dados_problema)

     MH  NJOBS  NMACHINES  CMAXOPT OBJFUNC  CMAX  TIMEEXEC  NUMGEN  NUMPART  \
0    GA     10          5      655      WT   837     10.62   100.0      NaN   
1    GA     10         10      887      WT  1504     13.50   100.0      NaN   
2    GA     10         10      887      WT  1653     13.71   100.0      NaN   
3    GA     15          5      890      WT  1020     27.41   100.0      NaN   
4    GA     10         10      899      WT  1501      9.55   100.0      NaN   
..   ..    ...        ...      ...     ...   ...       ...     ...      ...   
550  SA     50         10     2972    Cmax  5191      7.55     NaN      NaN   
551  SA     50         10     3104    Cmax  5495      5.27     NaN      NaN   
552  SA     10         10      848      WT  1037      0.22     NaN      NaN   
553  SA     10         10      943      WT  1078      0.27     NaN      NaN   
554  SA     10         10      944      WT  1411      0.24     NaN      NaN   

     NUMITERA  INITTEMP  NUMITERAK  SA_STOPCRIT  TA

In [2]:
#CRIAR O VETOR COM 3 ELEMENTOS - VARIÁVEL DEPENDENTE

def criar_vetor(row):
    if row['MH'] == 'GA':
        return [row['NUMGEN'], 0, 0]
    elif row['MH'] == 'PSO':
        return [row['NUMITERA'], row['NUMPART'], 0]
    elif row['MH'] == 'SA':
        return [row['SA_STOPCRIT'], row['NUMITERAK'], row['INITTEMP']]
    elif row['MH'] == 'TS':
        return [row['TS_STOPCRIT'], row['TABULISTLEN'], 0]
    else:
        return [0, 0, 0]  

dados_problema['vetor_parametros'] = dados_problema.apply(criar_vetor, axis=1)
print(dados_problema[['MH', 'vetor_parametros']])
parametros = np.vstack(dados_problema['vetor_parametros'].values)

     MH     vetor_parametros
0    GA        [100.0, 0, 0]
1    GA        [100.0, 0, 0]
2    GA        [100.0, 0, 0]
3    GA        [100.0, 0, 0]
4    GA        [100.0, 0, 0]
..   ..                  ...
550  SA  [190.0, 31.0, 15.0]
551  SA  [188.0, 29.0, 15.0]
552  SA   [33.0, 13.0, 14.0]
553  SA   [33.0, 13.0, 14.0]
554  SA   [38.0, 15.0, 14.0]

[555 rows x 2 columns]


In [3]:
#COLOCAR AS META-HEURÍSTICAS COMO VALORES NUMÉRICOS - NECESSÁRIO EM ALGUMAS TÉCNICAS DE MACHINE LEARNING
mh_map = {'GA': 0, 'PSO': 1, 'TS': 2, 'SA': 3}
dados_problema['MH_encoded'] = dados_problema['MH'].map(mh_map)

#VARIAVEIS INDEPENDENTES
X = dados_problema[['MH_encoded','NJOBS', 'NMACHINES', 'CMAXOPT', 'OBJFUNC']]

#COLUNA OBJFUNX: PASSAR CMAX = 0 E WT=1 - NECESSÁRIO EM ALGUMAS TÉCNICAS DE MACHINE LEARNING
X['OBJFUNC'] = X['OBJFUNC'].map({'Cmax': 0, 'WT': 1})

#VARIÉVEL DEPENDENTE 
Y = parametros


#DIVISÃO EM TREINO E TESTE - DIVISÃO ESTRATIFICADA POR MH
X_train, X_test, Y_train, Y_test, strat_train, strat_test = train_test_split(
    X, Y, dados_problema['MH'], 
    test_size=0.3, random_state=42, stratify=dados_problema['MH']
)

#PERCENTAGENS DE CADA MH NO CONJUNTO DE TREINO E TESTE - VERIFICA QUE A DIVISÃO ESTRATIFICADA FOI REALIZADA DE FORMA CORRETA
print("Distribuição de MH no conjunto de TREINO:")
print(strat_train.value_counts(normalize=True) * 100)
print("\nDistribuição de MH no conjunto de TESTE:")
print(strat_test.value_counts(normalize=True) * 100)

Distribuição de MH no conjunto de TREINO:
TS     38.659794
SA     30.154639
PSO    23.711340
GA      7.474227
Name: MH, dtype: float64

Distribuição de MH no conjunto de TESTE:
TS     38.922156
SA     29.940120
PSO    23.952096
GA      7.185629
Name: MH, dtype: float64


C:\Users\Ana Silva\AppData\Local\Temp\ipykernel_6544\3638300307.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['OBJFUNC'] = X['OBJFUNC'].map({'Cmax': 0, 'WT': 1})


In [ ]:
#ÁRVORE DE DECISÃO ---------------------------------------------------------------------------------------------------------------------------

#GARANTIR OS ZEROS NAS POSIÇÕES DO VETOR PARA CADA META-HEURÍSTICA 
def corrigir_predicao(mh, vetor_predito):
    vetor_predito = np.round(vetor_predito).astype(int)
    if mh == 'GA':
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = 0
        vetor_predito[2] = 0
    elif mh in ['TS', 'PSO']:
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = max(vetor_predito[1], 1)
        vetor_predito[2] = 0
    elif mh == 'SA':
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = max(vetor_predito[1], 1)
        vetor_predito[2] = max(vetor_predito[2], 1)
    return vetor_predito.tolist()

#CRIAR MODELO ÁRVORE DE DECISÃO
dt_regressor = DecisionTreeRegressor(random_state=42)
multi_output_regressor = MultiOutputRegressor(dt_regressor)

#DEFINIR PARÂMETROS 
param_grid = {
    'estimator__criterion': ['squared_error', 'absolute_error'], 
    'estimator__max_depth': [3, 5, 10, 15, 20, None],
    'estimator__min_samples_split': [2, 5, 10, 15],
    'estimator__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 10, 15]
}

#GRIDSEARCH COM VALIDAÇÃO CRUZADA
grid_search = GridSearchCV(
    multi_output_regressor, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
    )

#TREINAR O MODELO À PROCURA DOS MELHOR HIPERPARÂMETROS
grid_search.fit(X_train, Y_train)

#OBTER O MELHOR MODELO
best_model = grid_search.best_estimator_
print(f"Melhores parâmetros: {grid_search.best_params_}\n")

#PREVISÕES
Y_pred = best_model.predict(X_test)

#APLICAR A "CORREÇÃO"
Y_pred_corrigido = []
for mh, pred, real in zip(strat_test, Y_pred, Y_test):
    corrigido = corrigir_predicao(mh, pred)
    Y_pred_corrigido.append(corrigido)
    #print(f"MH: {mh} | Real: {real.tolist()} | Previsto: {corrigido}")

#CÁLCULAR MÉTRICAS DE AVALIAÇÃO
mse = mean_squared_error(Y_test, Y_pred_corrigido)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred_corrigido)
r2 = r2_score(Y_test, Y_pred_corrigido)

print("\nMétricas de Avaliação:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

#PARA O CÁLCULO DO MAPE - IGNORA AS DIVISÕES POR ZERO 
Y_test_np = np.array(Y_test)
Y_pred_corrigido_np = np.array(Y_pred_corrigido)
mape_por_parametro = []
for i in range(Y_test_np.shape[1]):
    y_true = Y_test_np[:, i]
    y_pred = Y_pred_corrigido_np[:, i]
    mask = y_true != 0  
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan  #SE TODOS FOREM ZERO, NÃO CALCULA 
    mape_por_parametro.append(mape)
mape_geral = np.nanmean(mape_por_parametro)     #MÉDIA
print("\nMAPE por Parâmetro:")
for i, mape_val in enumerate(mape_por_parametro):
    print(f"Parâmetro {i+1}: {mape_val:.2f}%")
print(f"MAPE Geral: {mape_geral:.2f}%")

#IMPORTANCIA DAS FEATURES - COLOCA NUM EXCEL POR PARÂMETRO (CADA POSIÇÃO DO VETOR É UM PARÂMETRO) E NOUTRO O GERAL 
print("\nImportância das Features por Parâmetro:")
feature_importances = []
feature_names = X_train.columns
df_importancias_por_parametro = pd.DataFrame()
for i, estimator in enumerate(best_model.estimators_):
    importance = estimator.feature_importances_
    feature_importances.append(importance)
    #CRIAR UMA COLUNA POR PARÂMETRO
    df_importancias_por_parametro[f'Parâmetro_{i+1}'] = pd.Series(importance, index=feature_names)
    print(f"\nParâmetro {i+1}:")
    for name, imp in zip(feature_names, importance):
        print(f"  {name}: {imp:.4f}")
#ARREDONDAR A 4 CASAS DECIMAIS 
df_importancias_por_parametro = df_importancias_por_parametro.round(4)
df_importancias_por_parametro.to_excel("importancia_por_parametro.xlsx", index=True)
feature_importances = np.array(feature_importances)
importance_geral = np.mean(feature_importances, axis=0)
df_importancia_geral = pd.DataFrame({
    "Feature": feature_names,
    "Importância_Média": np.round(importance_geral, 4)
})
df_importancia_geral.to_excel("importancia_geral.xlsx", index=False)
print("\nImportância Geral das Features (Média entre os parâmetros):")
for name, imp in zip(feature_names, importance_geral):
    print(f"  {name}: {imp:.4f}")

Melhores parâmetros: {'estimator__criterion': 'squared_error', 'estimator__max_depth': 5, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 5}


Métricas de Avaliação:
MSE: 3557.0479
RMSE: 59.6410
MAE: 11.7545
R²: 0.9762

MAPE por Parâmetro:
Parâmetro 1: 4.50%
Parâmetro 2: 3.60%
Parâmetro 3: 0.14%
MAPE Geral: 2.75%

Importância das Features por Parâmetro:

Parâmetro 1:
  MH_encoded: 0.7977
  NJOBS: 0.1938
  NMACHINES: 0.0000
  CMAXOPT: 0.0057
  OBJFUNC: 0.0027

Parâmetro 2:
  MH_encoded: 0.7808
  NJOBS: 0.0570
  NMACHINES: 0.0000
  CMAXOPT: 0.1622
  OBJFUNC: 0.0000

Parâmetro 3:
  MH_encoded: 1.0000
  NJOBS: 0.0000
  NMACHINES: 0.0000
  CMAXOPT: 0.0000
  OBJFUNC: 0.0000

Importância Geral das Features (Média entre os parâmetros):
  MH_encoded: 0.8595
  NJOBS: 0.0836
  NMACHINES: 0.0000
  CMAXOPT: 0.0560
  OBJFUNC: 0.0009


In [8]:
#RANDOM FOREST ------------------------------------------------------------------------------------------------------------------------------------------

#GARANTIR OS ZEROS NAS POSIÇÕES DO VETOR PARA CADA META-HEURÍSTICA  
def corrigir_predicao(mh, vetor_predito):
    vetor_predito = np.round(vetor_predito).astype(int)
    if mh == 'GA':
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = 0
        vetor_predito[2] = 0
    elif mh in ['TS', 'PSO']:
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = max(vetor_predito[1], 1)
        vetor_predito[2] = 0
    elif mh == 'SA':
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = max(vetor_predito[1], 1)
        vetor_predito[2] = max(vetor_predito[2], 1)
    return vetor_predito.tolist()

#CRIAR MODELO DE RANDOM FOREST
rf_regressor = RandomForestRegressor(random_state=42)
multi_output_regressor = MultiOutputRegressor(rf_regressor)

#DEFINIR PARÂMETROS
param_grid = {
    'estimator__n_estimators': [50, 100, 200, 300, 400],
    'estimator__max_depth': [3, 5, 7, 10, 20, 30, None],
    'estimator__min_samples_split': [2, 5, 10, 15],
    'estimator__min_samples_leaf': [1, 2, 4, 6],
    'estimator__max_features': [None, 'sqrt', 'log2'],
}

#GRIDSEARCH COM VALIDAÇÃO CRUZADA
grid_search = GridSearchCV(
    multi_output_regressor, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
)

#TREINAR O MODELO À PROCURA DOS MELHOR HIPERPARÂMETROS
grid_search.fit(X_train, Y_train)

#OBTER O MELHOR MODELO
best_model = grid_search.best_estimator_
print(f"Melhores parâmetros: {grid_search.best_params_}\n")

#PREVISÕES
Y_pred = best_model.predict(X_test)

#APLICAR A "CORREÇÃO"
Y_pred_corrigido = []
for mh, pred in zip(strat_test, Y_pred):
    Y_pred_corrigido.append(corrigir_predicao(mh, pred))
    #print(f"MH: {mh} | Real: {real.tolist()} | Previsto: {corrigido}") 

#CÁLCULAR AS MÉTRICAS DE AVALIAÇÃO 
mse = mean_squared_error(Y_test, Y_pred_corrigido)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred_corrigido)
r2 = r2_score(Y_test, Y_pred_corrigido)

print("\nMétricas de Avaliação:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

#PARA O CÁLCULO DO MAPE - IGNORA AS DIVISÕES POR ZERO 
Y_test_np = np.array(Y_test)
Y_pred_corrigido_np = np.array(Y_pred_corrigido)
mape_por_parametro = []
for i in range(Y_test_np.shape[1]):
    y_true = Y_test_np[:, i]
    y_pred = Y_pred_corrigido_np[:, i]
    mask = y_true != 0
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan
    mape_por_parametro.append(mape)
mape_geral = np.nanmean(mape_por_parametro)
print("\nMAPE por Parâmetro:")
for i, mape_val in enumerate(mape_por_parametro):
    print(f"Parâmetro {i+1}: {mape_val:.2f}%")
print(f"MAPE Geral: {mape_geral:.2f}%")

#IMPORTANCIA DAS FEATURES - COLOCA NUM EXCEL POR PARÂMETRO (CADA POSIÇÃO DO VETOR É UM PARÂMETRO) E NOUTRO O GERAL 
print("\nImportância das Features por Parâmetro:")
feature_importances = []
feature_names = X_train.columns
df_importancias_por_parametro = pd.DataFrame()
for i, estimator in enumerate(best_model.estimators_):
    importance = estimator.feature_importances_
    feature_importances.append(importance)
    #CRIAR UMA COLUNA POR PARÂMETRO
    df_importancias_por_parametro[f'Parâmetro_{i+1}'] = pd.Series(importance, index=feature_names)
    print(f"\nParâmetro {i+1}:")
    for name, imp in zip(feature_names, importance):
        print(f"  {name}: {imp:.4f}")
#ARREDONDAR A 4 CASAS DECIMAIS 
df_importancias_por_parametro = df_importancias_por_parametro.round(4)
df_importancias_por_parametro.to_excel("importancia_por_parametro.xlsx", index=True)
feature_importances = np.array(feature_importances)
importance_geral = np.mean(feature_importances, axis=0)
df_importancia_geral = pd.DataFrame({
    "Feature": feature_names,
    "Importância_Média": np.round(importance_geral, 4)
})
df_importancia_geral.to_excel("importancia_geral.xlsx", index=False)
print("\nImportância Geral das Features (Média entre os parâmetros):")
for name, imp in zip(feature_names, importance_geral):
    print(f"  {name}: {imp:.4f}")

Melhores parâmetros: {'estimator__max_depth': 5, 'estimator__max_features': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 5, 'estimator__n_estimators': 400}


Métricas de Avaliação:
MSE: 3515.9281
RMSE: 59.2953
MAE: 12.0319
R²: 0.9766

MAPE por Parâmetro:
Parâmetro 1: 4.79%
Parâmetro 2: 4.35%
Parâmetro 3: 0.14%
MAPE Geral: 3.09%

Importância das Features por Parâmetro:

Parâmetro 1:
  MH_encoded: 0.7962
  NJOBS: 0.1773
  NMACHINES: 0.0003
  CMAXOPT: 0.0251
  OBJFUNC: 0.0011

Parâmetro 2:
  MH_encoded: 0.7794
  NJOBS: 0.1144
  NMACHINES: 0.0001
  CMAXOPT: 0.1029
  OBJFUNC: 0.0033

Parâmetro 3:
  MH_encoded: 1.0000
  NJOBS: 0.0000
  NMACHINES: 0.0000
  CMAXOPT: 0.0000
  OBJFUNC: 0.0000

Importância Geral das Features (Média entre os parâmetros):
  MH_encoded: 0.8585
  NJOBS: 0.0972
  NMACHINES: 0.0001
  CMAXOPT: 0.0427
  OBJFUNC: 0.0015


In [ ]:
#REGRESSÃO LINEAR ---------------------------------------------------------------------------------------------------------------------------------------

#GARANTIR OS ZEROS NAS POSIÇÕES DO VETOR PARA CADA META-HEURÍSTICA  
def corrigir_predicao(mh, vetor_predito):
    vetor_predito = np.round(vetor_predito).astype(int)
    if mh == 'GA':
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = 0
        vetor_predito[2] = 0
    elif mh in ['TS', 'PSO']:
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = max(vetor_predito[1], 1)
        vetor_predito[2] = 0
    elif mh == 'SA':
        vetor_predito[0] = max(vetor_predito[0], 1)
        vetor_predito[1] = max(vetor_predito[1], 1)
        vetor_predito[2] = max(vetor_predito[2], 1)
    return vetor_predito.tolist()

#CRIAR MODELO DE REGRESSÃO LINEAR 
lr = LinearRegression()
multi_output_regressor = MultiOutputRegressor(lr)

#TREINAR O MODELO 
multi_output_regressor.fit(X_train, Y_train)

#PREVISÕES
Y_pred = multi_output_regressor.predict(X_test)

#APLICAR "CORREÇÃO"
Y_pred_corrigido = []
for mh, pred in zip(strat_test, Y_pred):
    Y_pred_corrigido.append(corrigir_predicao(mh, pred))
    #print(f"MH: {mh} | Real: {real.tolist()} | Previsto: {corrigido}") 

#CALCULAR MÉTRICAS DE AVALIAÇÃO
mse = mean_squared_error(Y_test, Y_pred_corrigido)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred_corrigido)
r2 = r2_score(Y_test, Y_pred_corrigido)

print("\nMétricas de Avaliação:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

#PARA O CÁLCULO DO MAPE - IGNORA AS DIVISÕES POR ZERO 
Y_test_np = np.array(Y_test)
Y_pred_corrigido_np = np.array(Y_pred_corrigido)
mape_por_parametro = []
for i in range(Y_test_np.shape[1]):
    y_true = Y_test_np[:, i]
    y_pred = Y_pred_corrigido_np[:, i]
    mask = y_true != 0
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan
    mape_por_parametro.append(mape)
mape_geral = np.nanmean(mape_por_parametro)
print("\nMAPE por Parâmetro:")
for i, mape_val in enumerate(mape_por_parametro):
    print(f"Parâmetro {i+1}: {mape_val:.2f}%")
print(f"MAPE Geral: {mape_geral:.2f}%")



Métricas de Avaliação:
MSE: 134857.3693
RMSE: 367.2293
MAE: 142.7425
R²: 0.4582

MAPE por Parâmetro:
Parâmetro 1: 204.35%
Parâmetro 2: 519.97%
Parâmetro 3: 25.76%
MAPE Geral: 250.03%


'\n#IMPORTANCIA DAS FEATURES - COLOCA NUM EXCEL\nprint("\nImportância das Features por Parâmetro:")\nfeature_importances = []\nfeature_names = X_train.columns\ndf_importancias_por_parametro = pd.DataFrame()\n\nfor i, estimator in enumerate(best_model.estimators_):\n    importance = estimator.feature_importances_\n    feature_importances.append(importance)\n\n    #CRIAR UMA COLUNA POR PARÂMETRO\n    df_importancias_por_parametro[f\'Parâmetro_{i+1}\'] = pd.Series(importance, index=feature_names)\n    print(f"\nParâmetro {i+1}:")\n    for name, imp in zip(feature_names, importance):\n        print(f"  {name}: {imp:.4f}")\n\n#ARREDONDAR A 4 CASAS DECIMAIS \ndf_importancias_por_parametro = df_importancias_por_parametro.round(4)\ndf_importancias_por_parametro.to_excel("importancia_por_parametro.xlsx", index=True)\nfeature_importances = np.array(feature_importances)\nimportance_geral = np.mean(feature_importances, axis=0)\n\ndf_importancia_geral = pd.DataFrame({\n    "Feature": feature_names,